# Import libraries

In [13]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd
from scipy import spatial

# Buildings

In [14]:
# Import Cambridge Shapefile Data 
shapefile_path='.././02 Data/Shapefile/Cambridge/'
buildings = gpd.read_file(shapefile_path+'Buildings.shp').to_crs(crs='EPSG:4326')
centroids =buildings.geometry.centroid
buildings['lon_POI']=[c.x for c in centroids]
buildings['lat_POI']=[c.y for c in centroids]

C:\Users\a9048\anaconda3\envs\new_mobility\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.


# Dataframes for input database

## Residential

In [3]:
# Generate de dataframe with the RESIDENTIAL locations
residential_categories=['Residential','Mixed Use Residential','Assisted Living/Boarding House','Education Residential']
residential_df=buildings[buildings['Category'].isin(residential_categories)]
drop_list=['ML','MAP','LOT','MixedUseTy','GQLodgingH','GradStuden','CondoFlag','TaxStatus','Shape_STAr','Shape_STLe','geometry',
          'Location','LandArea','LUCode','ExistUnits']
residential_df=residential_df.drop(drop_list,axis=1)
residential_df=residential_df.reset_index().drop('index',axis=1)
residential_df.head()

,LUDesc,Category,lon_POI,lat_POI
0,TWO-FAM-RES,Residential,-71.134264,42.402296
1,TWO-FAM-RES,Residential,-71.133773,42.402267
2,TWO-FAM-RES,Residential,-71.133621,42.402224
3,TWO-FAM-RES,Residential,-71.133483,42.402157
4,TWO-FAM-RES,Residential,-71.134353,42.402188


In [4]:
# Save the results of RESIDENTIAL locations
residential_path=Path('./Input Database/residentialbuildings.csv')
residential_df.to_csv(residential_path,index=False)

## Park

In [5]:
# Generate de dataframe with the PARK locations
park_categories=['Public Open Space','Privately-Owned Open Space']
park_df=buildings[buildings['Category'].isin(park_categories)]
drop_list=['ML','MAP','LOT','MixedUseTy','GQLodgingH','GradStuden','CondoFlag','TaxStatus','Shape_STAr','Shape_STLe','geometry',
          'Location','LandArea','LUCode','ExistUnits']
park_df=park_df.drop(drop_list,axis=1)
park_df=park_df.reset_index().drop('index',axis=1)
park_df.head()

,LUDesc,Category,lon_POI,lat_POI
0,Parklands,Public Open Space,-71.138390,42.399749
1,Parklands,Public Open Space,-71.132387,42.399878
2,Parklands,Public Open Space,-71.132720,42.399670
3,Parklands,Public Open Space,-71.132604,42.399613
4,Parklands,Public Open Space,-71.132471,42.397825


In [6]:
# Save the results of PARK locations
park_path=Path('./Input Database/parkbuildings.csv')
park_df.to_csv(park_path,index=False)

# Blockgroups

In [15]:
# Import CAMBRIDGE Shapefile Data 
shapefile_path='.././02 Data/Shapefile/Cambridge/'
blockgroups = gpd.read_file(shapefile_path+'DEMOGRAPHICS_BlockGroups2020.shp').to_crs(crs='EPSG:4326')
centroids =blockgroups.geometry.centroid
blockgroups['lon_POI']=[c.x for c in centroids]
blockgroups['lat_POI']=[c.y for c in centroids]

C:\Users\a9048\anaconda3\envs\new_mobility\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.


In [16]:
blockgroups_cambridge=blockgroups['GEOID20'].unique()

# Shapefile generation for GAMA

In [9]:
buildings.loc[(buildings.Category=='Residential') | (buildings.Category=='Mixed Use Residential') | (buildings.Category=='Education Residential') | (buildings.Category=='Assisted Living/Boarding House'), 'usage'] = "R"
buildings.loc[(buildings.Category=='Office') | (buildings.Category=='Industrial') | (buildings.Category=='Mixed Use Industrial') | (buildings.Category=='Office/R&D'), 'usage'] = "O"
buildings.loc[(buildings.Category=='Public Open Space') | (buildings.Category=='Privately-Owned Open Space'), 'usage'] = "P"
buildings.loc[(buildings.Category=='Health'), 'usage'] = "H"
buildings.loc[(buildings.Category=='Higher Education') | (buildings.Category=='Education') | (buildings.Category=='Mixed Use Education'), 'usage'] = "E"
        
buildings.head()

,ML,MAP,LOT,Location,LandArea,LUCode,LUDesc,Category,ExistUnits,MixedUseTy,GQLodgingH,GradStuden,CondoFlag,TaxStatus,Shape_STAr,Shape_STLe,geometry,lon_POI,lat_POI,usage
0,185-42,185,42,29 ALEWIFE BROOK PKWY,5323.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,5349.684326,318.529860,"POLYGON ((-71.13403 42.40230, -71.13408 42.402...",-71.134264,42.402296,R
1,185-48,185,48,70 MATIGNON RD,4254.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,4104.788330,278.569316,"POLYGON ((-71.13360 42.40236, -71.13380 42.402...",-71.133773,42.402267,R
2,185-49,185,49,66 MATIGNON RD,4703.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,4442.930176,293.858467,"POLYGON ((-71.13344 42.40232, -71.13366 42.402...",-71.133621,42.402224,R
3,185-55,185,55,58 MATIGNON RD,4777.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,4993.306641,305.455373,"POLYGON ((-71.13331 42.40223, -71.13353 42.402...",-71.133483,42.402157,R
4,185-43,185,43,27 ALEWIFE BROOK PKWY,5105.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,4743.144287,304.957587,"POLYGON ((-71.13413 42.40218, -71.13423 42.402...",-71.134353,42.402188,R


In [11]:
buildings.loc[(buildings.Category=='NaN'), 'usage'] = "Other"

,ML,MAP,LOT,Location,LandArea,LUCode,LUDesc,Category,ExistUnits,MixedUseTy,GQLodgingH,GradStuden,CondoFlag,TaxStatus,Shape_STAr,Shape_STLe,geometry,lon_POI,lat_POI,usage
0,185-42,185,42,29 ALEWIFE BROOK PKWY,5323.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,5349.684326,318.529860,"POLYGON ((-71.13403 42.40230, -71.13408 42.402...",-71.134264,42.402296,R
1,185-48,185,48,70 MATIGNON RD,4254.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,4104.788330,278.569316,"POLYGON ((-71.13360 42.40236, -71.13380 42.402...",-71.133773,42.402267,R
2,185-49,185,49,66 MATIGNON RD,4703.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,4442.930176,293.858467,"POLYGON ((-71.13344 42.40232, -71.13366 42.402...",-71.133621,42.402224,R
3,185-55,185,55,58 MATIGNON RD,4777.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,4993.306641,305.455373,"POLYGON ((-71.13331 42.40223, -71.13353 42.402...",-71.133483,42.402157,R
4,185-43,185,43,27 ALEWIFE BROOK PKWY,5105.0,104,TWO-FAM-RES,Residential,2.0,None,None,None,None,None,4743.144287,304.957587,"POLYGON ((-71.13413 42.40218, -71.13423 42.402...",-71.134353,42.402188,R


In [12]:
buildings.to_file(filename='.././02 Data/Shapefile/Cambridge/Buildings', driver='ESRI Shapefile')